In [26]:
import os 
from dotenv import load_dotenv 
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, trim_messages
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

load_dotenv()

True

In [10]:
store = {}
def get_message_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]  

In [6]:
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=os.getenv('GROQ_API_KEY'))

In [ ]:
# Automtatically injects past messages and persists new ones based on session_id from config
with_message_history = RunnableWithMessageHistory(model, get_message_history)

config = {"configurable": {"session_id": "chat1"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi, I'm a MERN stack developer learning AI/ML.")],
    config=config
)
response

AIMessage(content="That's fantastic!  It's great to see a MERN stack developer expanding their skillset into the world of AI/ML. Your background in web development will be a real asset as you learn these new technologies. \n\nHere are some thoughts to get you started:\n\n**Bridging the Gap:**\n\n* **Data is Key:**  AI/ML thrives on data.  You'll likely need to learn about data cleaning, preprocessing, and manipulation, which can complement your existing skills in handling data on the web.\n* **Backend Power:** Your MERN stack experience with Node.js and databases will be valuable for building the backend infrastructure to support AI/ML models.\n\n**Areas to Explore:**\n\n* **Computer Vision for Web Apps:** Imagine adding image recognition to your projects.  You could build features like:\n    * Product tagging and search\n    * Content moderation\n    * Image-based filters\n* **Natural Language Processing (NLP):**  Power up your web applications with:\n    * Chatbots and conversational

In [20]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions in {language} to the best of your abilities."),
    MessagesPlaceholder(variable_name="messages")
])
chain = prompt | model
with_message_history = RunnableWithMessageHistory(chain, get_message_history, input_messages_key="messages")

config = {"configurable": {"session_id": "chat2"}}
response = with_message_history.invoke(
    {
        "messages": [HumanMessage("Hi, I'm a Java developer")], 
        "language": "Hindi"
    },
    config=config
)
response

AIMessage(content='नमस्ते!  \n\nयह जानकर अच्छा लगा कि आप एक Java डेवलपर हैं!  \n\nआप क्या जानकारी या मदद चाहते हैं?  मैं निम्न में आपकी सहायता कर सकता हूँ:\n\n* **Java अवधारणाओं को समझाना:**  पॉलीमॉर्फिज्म या जनरिक्स के बारे में रिफ्रेशर चाहिए?\n* **कोड डीबगिंग:**  कोई मुश्किल बग़ में फंसे हैं? मैं आपको समस्या का समाधान खोजने में मदद करने की कोशिश कर सकता हूँ।\n* **कोड स्निपेट जनरेट करना:**  किसी विशिष्ट कार्य के लिए त्वरित कोड की आवश्यकता है?\n* **बेस्ट प्रैक्टिस पर चर्चा करना:**  आधुनिक Java डेवलपमेंट तकनीकों के बारे में जानना चाहते हैं?\n* **संसाधन खोजने:**  ट्यूटोरियल, दस्तावेज़ या लाइब्रेरी की तलाश है?\n\n\n बस मुझे बताएं कि आप क्या सोच रहे हैं! 😊 \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 248, 'prompt_tokens': 196, 'total_tokens': 444, 'completion_time': 0.450909091, 'prompt_time': 0.004545124, 'queue_time': 0.252167555, 'total_time': 0.455454215}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_de

### Managing conversation history

In [33]:
messages = [
    SystemMessage(content="You're a helpful assistant who can answer all questions"),
    HumanMessage(content="Hi, I'm Bob"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
    HumanMessage(content="What is the capital of India?"),
    AIMessage(content="New Delhi"),
]
trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
trimmer.invoke(messages)

[SystemMessage(content="You're a helpful assistant who can answer all questions", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the capital of India?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='New Delhi', additional_kwargs={}, response_metadata={})]

In [ ]:
""" 
Input comes in (likely {"messages": [chat_history], "user": ...}).
RunnablePassthrough.assign → modifies "messages" by trimming old/long parts.
Passes modified data to prompt, which structures it into a text prompt.
That structured prompt goes into the model.
Model generates the answer.
"""
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer) | 
    prompt | 
    model
)
with_message_history = RunnableWithMessageHistory(
    chain, get_message_history,
    input_messages_key="messages"
)
config={"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    {
    "messages": messages + [HumanMessage("What is the capital of Ukraine?")],
    "language": "English"
    }, 
    config=config
)
response

AIMessage(content="2 + 3 = 5  \n\nIt's like adding two friends to three friends. You get a group of five friends! 🥳\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 80, 'total_tokens': 112, 'completion_time': 0.058181818, 'prompt_time': 0.00250301, 'queue_time': 0.25179336, 'total_time': 0.060684828}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a5b5a119-77e4-405b-8f25-952cb3538429-0', usage_metadata={'input_tokens': 80, 'output_tokens': 32, 'total_tokens': 112})

In [38]:
response = with_message_history.invoke(
    {
    "messages": messages + [HumanMessage("What was the last Math problem which I asked?")],
    "language": "English"
    }, 
    config=config
)
response

AIMessage(content="As a helpful and harmless AI assistant, I have no memory of past conversations. Every interaction we have is fresh and new!\n\nIf you'd like to ask me a math problem now, I'm happy to help. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 68, 'total_tokens': 120, 'completion_time': 0.094545455, 'prompt_time': 0.002278689, 'queue_time': 0.250143582, 'total_time': 0.096824144}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--8b9271c8-319d-4487-aa5a-49361a8ef394-0', usage_metadata={'input_tokens': 68, 'output_tokens': 52, 'total_tokens': 120})